# Dimensión: Movilidad y transporte

In [1]:
import pandas as pd
import geopandas as gpd
from functools import reduce

# Datos

In [2]:
# carga de la capa de gob locales
df_goblocales = gpd.read_file("../GobiernosLocales/GobiernosLocales.shp")
df_goblocales.rename(columns={'Provincia':'provincia', 'Departamen':'departamen', 'Municipio':'municipio', 'UTA2020':'uta2020'}, inplace=True)
# cálculo de superficie
df_goblocales = df_goblocales.to_crs("EPSG:3035")
df_goblocales['area_total_k2'] = df_goblocales.area / 1000000
df_goblocales = df_goblocales.to_crs("EPSG:4326")

In [3]:
# carga de datos de las dimensiones inserción territorial y sistema demográfico
df_sup_data = gpd.read_file("../CapasGeograficasOriginales/InsercionTerritorial_GobiernosLocales.shp")
df_pob_data = pd.read_csv("../BD_Originales/SistemaDemografico_GobiernosLocales.csv")

In [4]:
 # carga de las capas de redes viales 
df_red_nac = gpd.read_file("../CapasGeograficasOriginales/RedVialNacional.geojson")
df_red_prov = gpd.read_file("../CapasGeograficasOriginales/RedVialProvincial.geojson")
df_red_terc = gpd.read_file("../CapasGeograficasOriginales/RedVialTerciaria.geojson")
df_red_urb = gpd.read_file("../CapasGeograficasOriginales/RedVialUrbana.geojson")

In [5]:
# importación de los datos de estaciones y redes de FFCC
df_estFFCC = gpd.read_file("../CapasGeograficasOriginales/EstacionFFCC.geojson")
df_redFFCC = gpd.read_file("../CapasGeograficasOriginales/RedFFCC.geojson")

In [6]:
# importación de la capa de estaciones de omnibus, puetos, aeropuertos, etc
df_estaciones = gpd.read_file("../CapasGeograficasOriginales/EstacionOmnibus.geojson")
df_aerodromo = gpd.read_file("../CapasGeograficasOriginales/aerodromo.geojson")
df_aeropuerto = gpd.read_file("../CapasGeograficasOriginales/aeropuerto.geojson")
df_helipuerto = gpd.read_file("../CapasGeograficasOriginales/Helipuerto.geojson")
df_puertos = gpd.read_file("../CapasGeograficasOriginales/Puertos.geojson")

In [7]:
def acortar_lista_nombres(listanombres):
    if len(listanombres) >= 250:
        while len(listanombres) >= 250:
            x = eval(listanombres)[:-1]
            listanombres = str(list(set(','.join(x).split(','))))
        return str(list(set(','.join(eval(listanombres)+['...']).split(',')))) 
    else:
        return listanombres

# Accesos y Redes Viales

## Rutas nacionales

In [8]:
# creación de un campo de nombre de ruta
df_red_nac['ruta_name'] = df_red_nac.typ +' '+ df_red_nac.hct +' '+ df_red_nac.rtn
df_red_nac_exploded = df_red_nac.explode()

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# cruce de las rutas nacionales con los gobiernos locales
df_inter_gob_nac = gpd.overlay(df_red_nac_exploded, df_goblocales, how='union').explode().reset_index(drop=True)
df_inter_gob_nac = df_inter_gob_nac.loc[df_inter_gob_nac.uta2020.notnull()].reset_index()
# calculo de la longitud de las líneas
df_inter_gob_nac = df_inter_gob_nac.to_crs("EPSG:3035")
df_inter_gob_nac['long_km'] = df_inter_gob_nac.length / 1000
df_inter_gob_nac = df_inter_gob_nac.to_crs("EPSG:4326")
# exrpotación del resultado
df_inter_gob_nac.to_file("../CapasGeograficasCruzadas/rutasNacionales_goblocales.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 2326 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  if __name__ == '__main__':


In [10]:
# Generación de los valores de las distintas variable
# Cantidad de rutas
df_inter_gob_nac_cantidad = df_inter_gob_nac.groupby(['uta2020'])['ruta_name'].apply(lambda x: len(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_nac_cantidad.rename(columns = {'ruta_name':'nac_cant'}, inplace=True)
# Lista de nombres de rutas
df_inter_gob_nac_name = df_inter_gob_nac.groupby(['uta2020'])['ruta_name'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_nac_name.rename(columns = {'ruta_name':'nac_list'}, inplace=True)
df_inter_gob_nac_name['nac_list'] = df_inter_gob_nac_name.apply(lambda row: acortar_lista_nombres(row.nac_list), axis=1)
# Longitud total de rutas
df_inter_gob_nac_long = df_inter_gob_nac.groupby(['uta2020'])['long_km'].sum().reset_index()
df_inter_gob_nac_long.rename(columns = {'long_km':'nac_long_km'}, inplace=True)
# Longitud por material de ruta
df_inter_gob_nac_long_mat = df_inter_gob_nac.pivot_table(index='uta2020', columns='rst', values='long_km', aggfunc=sum).fillna(0).reset_index()
df_inter_gob_nac_long_mat.rename(columns = {df_inter_gob_nac_long_mat.columns[1]:'nac_mat_'+df_inter_gob_nac_long_mat.columns[1].lower(),
    df_inter_gob_nac_long_mat.columns[2]:'nac_mat_'+df_inter_gob_nac_long_mat.columns[2].lower(),
    df_inter_gob_nac_long_mat.columns[3]:'nac_mat_'+df_inter_gob_nac_long_mat.columns[3].lower()}, inplace=True)
# Longitud por tipo de ruta
df_inter_gob_nac_typ = df_inter_gob_nac.pivot_table(index='uta2020', columns='typ', values='long_km', aggfunc=sum).fillna(0).reset_index()
df_inter_gob_nac_typ.rename(columns = {df_inter_gob_nac_typ.columns[1]:'nac_typ_'+df_inter_gob_nac_typ.columns[1].lower(),
    df_inter_gob_nac_typ.columns[2]:'nac_typ_'+df_inter_gob_nac_typ.columns[2].lower(),
    df_inter_gob_nac_typ.columns[3]:'nac_typ_'+df_inter_gob_nac_typ.columns[3].lower()}, inplace=True)

In [11]:
# generación del dataframe de rutas nacionales
dfs_nac = [df_inter_gob_nac_cantidad, df_inter_gob_nac_name, df_inter_gob_nac_long, df_inter_gob_nac_long_mat, df_inter_gob_nac_typ]
dfs_nac_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_nac)
# cambio del nombre de las columnas
dfs_nac_merged.rename(columns={'nac_cant':'u_nac_tot', 'nac_list':'st_nac_nam', 'nac_long_km':'km_nac_tot', 'nac_mat_consolidada':'km_mna_con',
            'nac_mat_pavimentada':'km_mna_pav', 'nac_mat_tierra':'km_mna_tie', 'nac_typ_autopista':'km_tna_aut', 'nac_typ_autovía':'km_tna_auv',
            'nac_typ_ruta':'km_tna_rut'}, inplace=True)
# formateo de los campos numéricos
dfs_nac_merged.km_nac_tot = round(dfs_nac_merged.km_nac_tot, 3)
dfs_nac_merged.km_mna_con = round(dfs_nac_merged.km_mna_con, 3)
dfs_nac_merged.km_mna_pav = round(dfs_nac_merged.km_mna_pav, 3)
dfs_nac_merged.km_mna_tie = round(dfs_nac_merged.km_mna_tie, 3)
dfs_nac_merged.km_tna_aut = round(dfs_nac_merged.km_tna_aut, 3)
dfs_nac_merged.km_tna_auv = round(dfs_nac_merged.km_tna_auv, 3)
dfs_nac_merged.km_tna_rut = round(dfs_nac_merged.km_tna_rut, 3)

dfs_nac_merged.head()

,uta2020,u_nac_tot,st_nac_nam,km_nac_tot,km_mna_con,km_mna_pav,km_mna_tie,km_tna_aut,km_tna_auv,km_tna_rut
0,020072007,1,['Autopista Nacional 1'],0.120,0.0,0.120,0.0,0.120,0.0,0.000
1,020282028,2,"['Autopista Nacional 1', 'Ruta Nacional A001']",7.012,0.0,7.012,0.0,4.934,0.0,2.078
2,020562056,2,"['Autopista Nacional A002', 'Autopista Naciona...",16.196,0.0,16.196,0.0,16.196,0.0,0.000
3,020632063,2,"['Autopista Nacional 7', 'Autopista Nacional A...",16.160,0.0,16.160,0.0,16.160,0.0,0.000
4,020702070,2,"['Autopista Nacional 7', 'Autopista Nacional A...",5.949,0.0,5.949,0.0,5.949,0.0,0.000


## Rutas provinciales

In [12]:
# creación de un campo de nombre de ruta y limpieza del campo "rst"
df_red_prov['ruta_name'] = "Ruta Pcial. "+ df_red_prov.nr.astype(str)
df_red_prov.loc[df_red_prov.ruta_name.isnull(), 'ruta_name'] = "Ruta Pcial."
df_red_prov.rst = df_red_prov.rst.astype(str)
df_red_prov.rst = df_red_prov.apply(lambda row: row.rst.split('.')[0], axis=1)
# limpieza del campo "typ"
df_red_prov.loc[df_red_prov.typ == 53, 'typ'] = 47

df_red_prov_exploded = df_red_prov.explode()

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  if __name__ == '__main__':


In [13]:
# cruce de las rutas provinciales con los gobiernos locales
df_inter_gob_prov = gpd.overlay(df_red_prov_exploded, df_goblocales, how='union').explode().reset_index(drop=True)
df_inter_gob_prov = df_inter_gob_prov.loc[df_inter_gob_prov.uta2020.notnull()].reset_index()
# calculo de la longitud de las líneas
df_inter_gob_prov = df_inter_gob_prov.to_crs("EPSG:3035")
df_inter_gob_prov['long_km'] = df_inter_gob_prov.length / 1000
df_inter_gob_prov = df_inter_gob_prov.to_crs("EPSG:4326")
# exrpotación del resultado
df_inter_gob_prov.to_file("../CapasGeograficasCruzadas/rutasProvinciales_goblocales.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 2326 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  if __name__ == '__main__':


In [14]:
# Generación de los valores de las distintas variable
# Cantidad de rutas
df_inter_gob_prov_cantidad = df_inter_gob_prov.groupby(['uta2020'])['ruta_name'].apply(lambda x: len(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_prov_cantidad.rename(columns = {'ruta_name':'prov_cant'}, inplace=True)
# Lista de nombres de rutas
df_inter_gob_prov_name = df_inter_gob_prov.groupby(['uta2020'])['ruta_name'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_prov_name.rename(columns = {'ruta_name':'prov_list'}, inplace=True)
df_inter_gob_prov_name['prov_list'] = df_inter_gob_prov_name.apply(lambda row: acortar_lista_nombres(row.prov_list), axis=1)
# Longitud total de rutas
df_inter_gob_prov_long = df_inter_gob_prov.groupby(['uta2020'])['long_km'].sum().reset_index()
df_inter_gob_prov_long.rename(columns = {'long_km':'prov_long_km'}, inplace=True)
# Longitud por material de ruta
df_inter_gob_prov_long_mat = df_inter_gob_prov.pivot_table(index='uta2020', columns='rst', values='long_km', aggfunc=sum).fillna(0).reset_index()
df_inter_gob_prov_long_mat.rename(columns = {'-1':'prov_mat_sininfo',
    '1':'prov_mat_pavimentado', '13':'prov_mat_consolidado', '19':'prov_mat_tierra'}, inplace=True)
# Longitud por tipo de ruta
df_inter_gob_prov_typ = df_inter_gob_prov.pivot_table(index='uta2020', columns='typ', values='long_km', aggfunc=sum).fillna(0).reset_index()
df_inter_gob_prov_typ.rename(columns = {-1.0:'prov_typ_sininfo',
    41.0:'prov_typ_autopista', 47.0:'prov_typ_autovía', 51.0:'prov_typ_caminouncarril', 54.0:'prov_typ_caminodoscarriles'}, inplace=True)

In [15]:
# Limpieza de columnas de tipo
df_inter_gob_prov_typ.loc[df_inter_gob_prov_typ[0.0] != 0, 'prov_typ_sininfo'] = df_inter_gob_prov_typ.loc[df_inter_gob_prov_typ[0.0] != 0, 'prov_typ_sininfo'] + df_inter_gob_prov_typ.loc[df_inter_gob_prov_typ[0.0] != 0, 0.0]
df_inter_gob_prov_typ.loc[df_inter_gob_prov_typ[-2.0] != 0, 'prov_typ_sininfo'] = df_inter_gob_prov_typ.loc[df_inter_gob_prov_typ[-2.0] != 0, 'prov_typ_sininfo'] + df_inter_gob_prov_typ.loc[df_inter_gob_prov_typ[-2.0] != 0, -2.0]
df_inter_gob_prov_typ.drop(columns=[0.0, -2.0], inplace=True)

In [16]:
# generación del dataframe de rutas provinciales
dfs_prov = [df_inter_gob_prov_cantidad, df_inter_gob_prov_name, df_inter_gob_prov_long, df_inter_gob_prov_long_mat, df_inter_gob_prov_typ]
dfs_prov_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_prov)
# cambio del nombre de las columnas
dfs_prov_merged.rename(columns={'prov_cant':'u_pro_tot', 'prov_list':'st_pro_nam', 'prov_long_km':'km_pro_tot', 'prov_mat_sininfo':'km_mpr_sin',
            'prov_mat_pavimentado':'km_mpr_pav', 'prov_mat_consolidado':'km_mpr_con', 'prov_mat_tierra':'km_mpr_tie', 'prov_typ_sininfo':'km_tpr_sin', 
            'prov_typ_autopista':'km_tpr_aut', 'prov_typ_autovía':'km_tpr_auv', 'prov_typ_caminouncarril':'km_tpr_cuc', 
            'prov_typ_caminodoscarriles':'km_tpr_cdc'}, inplace=True)
# formateo de los campos numéricos
dfs_prov_merged.km_pro_tot = round(dfs_prov_merged.km_pro_tot, 3)
dfs_prov_merged.km_mpr_sin = round(dfs_prov_merged.km_mpr_sin, 3)
dfs_prov_merged.km_mpr_pav = round(dfs_prov_merged.km_mpr_pav, 3)
dfs_prov_merged.km_mpr_con = round(dfs_prov_merged.km_mpr_con, 3)
dfs_prov_merged.km_mpr_tie = round(dfs_prov_merged.km_mpr_tie, 3)
dfs_prov_merged.km_tpr_sin = round(dfs_prov_merged.km_tpr_sin, 3)
dfs_prov_merged.km_tpr_aut = round(dfs_prov_merged.km_tpr_aut, 3)
dfs_prov_merged.km_tpr_auv = round(dfs_prov_merged.km_tpr_auv, 3)
dfs_prov_merged.km_tpr_cuc = round(dfs_prov_merged.km_tpr_cuc, 3)
dfs_prov_merged.km_tpr_cdc = round(dfs_prov_merged.km_tpr_cdc, 3)

dfs_prov_merged.head()

,uta2020,u_pro_tot,st_pro_nam,km_pro_tot,km_mpr_sin,km_mpr_pav,km_mpr_con,km_mpr_tie,km_tpr_sin,km_tpr_aut,km_tpr_auv,km_tpr_cuc,km_tpr_cdc
0,020072007,1,['Ruta Pcial. AU1'],2.338,0.0,2.338,0.000,0.000,0.0,2.338,0.0,0.000,0.000
1,020562056,1,['Ruta Pcial. A002'],0.088,0.0,0.088,0.000,0.000,0.0,0.088,0.0,0.000,0.000
2,020632063,1,['Ruta Pcial. 7'],0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000
3,020912091,1,['Ruta Pcial. 27'],0.059,0.0,0.059,0.000,0.000,0.0,0.000,0.0,0.059,0.000
4,060070007,4,"['Ruta Pcial. 67', 'Ruta Pcial. 65', 'Ruta Pci...",335.493,0.0,140.906,13.488,181.099,0.0,0.000,0.0,13.488,322.005


## Caminos Terciarios

In [17]:
# cruce de las caminos terciarios con los gobiernos locales
df_inter_gob_terc = gpd.overlay(df_red_terc, df_goblocales, how='union').explode().reset_index(drop=True)
df_inter_gob_terc = df_inter_gob_terc.loc[df_inter_gob_terc.uta2020.notnull()].reset_index()
# calculo de la longitud de las líneas
df_inter_gob_terc = df_inter_gob_terc.to_crs("EPSG:3035")
df_inter_gob_terc['long_km'] = df_inter_gob_terc.length / 1000
df_inter_gob_terc = df_inter_gob_terc.to_crs("EPSG:4326")
# exrpotación del resultado
df_inter_gob_terc.to_file("../CapasGeograficasCruzadas/rutasTerciarias_goblocales.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 2326 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  if __name__ == '__main__':


In [18]:
# Generación de los valores de las distintas variable
# Longitud total de rutas
df_inter_gob_terc_long = df_inter_gob_terc.groupby(['uta2020'])['long_km'].sum().reset_index()
df_inter_gob_terc_long.rename(columns = {'long_km':'terc_long_km'}, inplace=True)
# Longitud por material de ruta
df_inter_gob_terc_long_mat = df_inter_gob_terc.pivot_table(index='uta2020', columns='rst', values='long_km', aggfunc=sum).fillna(0).reset_index()
df_inter_gob_terc_long_mat.rename(columns = {-1.0:'terc_mat_sininfo',
    1.0:'terc_mat_pavimentado', 13.0:'terc_mat_consolidado', 19.0:'terc_mat_tierra'}, inplace=True)
# Longitud por tipo de ruta
df_inter_gob_terc_typ = df_inter_gob_terc.pivot_table(index='uta2020', columns='typ', values='long_km', aggfunc=sum).fillna(0).reset_index()
df_inter_gob_terc_typ.rename(columns = {-1.0:'terc_typ_sininfo',
    41.0:'terc_typ_autopista', 47.0:'terc_typ_autovía', 51.0:'terc_typ_caminouncarril', 53.0:'terc_typ_viadeautovia', 54.0:'terc_typ_caminodoscarriles'}, inplace=True)

In [19]:
# Limpieza de columnas de tipo
df_inter_gob_terc_typ.loc[df_inter_gob_terc_typ[-2.0] != 0, 'terc_typ_sininfo'] = df_inter_gob_terc_typ.loc[df_inter_gob_terc_typ[-2.0] != 0, 'terc_typ_sininfo'] + df_inter_gob_terc_typ.loc[df_inter_gob_terc_typ[-2.0] != 0, -2.0]
df_inter_gob_terc_typ.drop(columns=[-2.0], inplace=True)

In [20]:
# generación del dataframe de caminos terciarios
dfs_terc = [df_inter_gob_terc_long, df_inter_gob_terc_long_mat, df_inter_gob_terc_typ]
dfs_terc_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_terc)
# cambio del nombre de las columnas
dfs_terc_merged.rename(columns={'terc_long_km':'km_ter_tot', 'terc_mat_sininfo':'km_mte_sin',
            'terc_mat_pavimentado':'km_mte_pav', 'terc_mat_consolidado':'km_mte_con', 'terc_mat_tierra':'km_mte_tie', 'terc_typ_sininfo':'km_tte_sin', 
            'terc_typ_autopista':'km_tte_aut', 'terc_typ_autovía':'km_tte_auv', 'terc_typ_caminouncarril':'km_tte_cuc', 'terc_typ_viadeautovia':'km_tte_vav',
            'terc_typ_caminodoscarriles':'km_tte_cdc'}, inplace=True)
# formateo de los campos numéricos
dfs_terc_merged.km_ter_tot = round(dfs_terc_merged.km_ter_tot, 3)
dfs_terc_merged.km_mte_sin = round(dfs_terc_merged.km_mte_sin, 3)
dfs_terc_merged.km_mte_pav = round(dfs_terc_merged.km_mte_pav, 3)
dfs_terc_merged.km_mte_con = round(dfs_terc_merged.km_mte_con, 3)
dfs_terc_merged.km_mte_tie = round(dfs_terc_merged.km_mte_tie, 3)
dfs_terc_merged.km_tte_sin = round(dfs_terc_merged.km_tte_sin, 3)
dfs_terc_merged.km_tte_aut = round(dfs_terc_merged.km_tte_aut, 3)
dfs_terc_merged.km_tte_auv = round(dfs_terc_merged.km_tte_auv, 3)
dfs_terc_merged.km_tte_cuc = round(dfs_terc_merged.km_tte_cuc, 3)
dfs_terc_merged.km_tte_cdc = round(dfs_terc_merged.km_tte_cdc, 3)

dfs_terc_merged.head()

,uta2020,km_ter_tot,km_mte_sin,km_mte_pav,km_mte_con,km_mte_tie,km_tte_sin,km_tte_aut,km_tte_auv,km_tte_cuc,km_tte_cdc
0,020072007,21.191,0.0,21.191,0.0,0.0,0.0,0.000,0.0,15.884,5.306
1,020142014,7.202,0.0,7.202,0.0,0.0,0.0,0.000,0.0,7.202,0.000
2,020212021,8.075,0.0,8.075,0.0,0.0,0.0,0.000,0.0,2.992,5.082
3,020282028,6.081,0.0,6.081,0.0,0.0,0.0,0.746,0.0,5.335,0.000
4,020352035,8.816,0.0,8.816,0.0,0.0,0.0,0.000,0.0,5.498,3.318


## Caminos Urbanos

In [21]:
# cruce de las caminos urbanos con los gobiernos locales
df_inter_gob_urb = gpd.overlay(df_red_urb, df_goblocales, how='union').explode().reset_index(drop=True)
df_inter_gob_urb = df_inter_gob_urb.loc[df_inter_gob_urb.uta2020.notnull()].reset_index()
# calculo de la longitud de las líneas
df_inter_gob_urb = df_inter_gob_urb.to_crs("EPSG:3035")
df_inter_gob_urb['long_km'] = df_inter_gob_urb.length / 1000
df_inter_gob_urb = df_inter_gob_urb.to_crs("EPSG:4326")
# exrpotación del resultado
df_inter_gob_urb.to_file("../CapasGeograficasCruzadas/callesUrbanas_goblocales.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 2326 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  if __name__ == '__main__':


In [22]:
# Generación de los valores de las distintas variable
# Longitud total de caminos
df_inter_gob_urb_long = df_inter_gob_urb.groupby(['uta2020'])['long_km'].sum().reset_index()
df_inter_gob_urb_long.rename(columns = {'long_km':'urb_long_km'}, inplace=True)
# Longitud por material de camino
df_inter_gob_urb_long_mat = df_inter_gob_urb.pivot_table(index='uta2020', columns='rst', values='long_km', aggfunc=sum).fillna(0).reset_index()
df_inter_gob_urb_long_mat.rename(columns = {-1.0:'urb_mat_sininfo', 0.0:'urb_mat_0',
    1.0:'urb_mat_pavimentado', 13.0:'urb_mat_consolidado', 19.0:'urb_mat_tierra'}, inplace=True)

In [23]:
# generación del dataframe de caminos terciarios
dfs_urb = [df_inter_gob_urb_long, df_inter_gob_urb_long_mat]
dfs_urb_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_urb)
# cambio del nombre de las columnas
dfs_urb_merged.rename(columns={'urb_long_km':'km_urb_tot', 'urb_mat_0':'km_mur_0',
            'urb_mat_pavimentado':'km_mur_pav', 'urb_mat_consolidado':'km_mur_con', 'urb_mat_tierra':'km_mur_tie'}, inplace=True)
# formateo de los campos numéricos
dfs_urb_merged.km_urb_tot = round(dfs_urb_merged.km_urb_tot, 3)
dfs_urb_merged.km_mur_0 = round(dfs_urb_merged.km_mur_0, 3)
dfs_urb_merged.km_mur_pav = round(dfs_urb_merged.km_mur_pav, 3)
dfs_urb_merged.km_mur_con = round(dfs_urb_merged.km_mur_con, 3)
dfs_urb_merged.km_mur_tie = round(dfs_urb_merged.km_mur_tie, 3)

dfs_urb_merged.head()

,uta2020,km_urb_tot,km_mur_0,km_mur_pav,km_mur_con,km_mur_tie
0,020072007,296.978,296.978,0.0,0.0,0.0
1,020142014,108.495,108.495,0.0,0.0,0.0
2,020212021,142.542,142.542,0.0,0.0,0.0
3,020282028,442.608,442.608,0.0,0.0,0.0
4,020352035,153.755,153.755,0.0,0.0,0.0


## Superficie y Habitantes

In [24]:
# lmipieza de los datos de población y superficie
df_pob_data.rename(columns={'coduta':'uta2020', 'dem_pobt':'totalpobl'}, inplace=True)

df_pob_data.uta2020 = df_pob_data.uta2020.astype(str)
for i in range(len(df_pob_data)):
    if len(df_pob_data.loc[i, 'uta2020']) == 8:
        df_pob_data.loc[i, 'uta2020'] = '0'+df_pob_data.loc[i, 'uta2020']

In [25]:
# limpieza del campo UTA2020
dfs_nac_merged.uta2020 = dfs_nac_merged.uta2020.astype(str)
for i in range(len(dfs_nac_merged)):
    if len(dfs_nac_merged.loc[i, 'uta2020']) == 8:
        dfs_nac_merged.loc[i, 'uta2020'] = '0'+dfs_nac_merged.loc[i, 'uta2020']
# unión de la tabla con datos poblacional y la capa de datos de rutas nacionales
dfs_nac_merged_suphab = dfs_nac_merged.merge(df_pob_data[['uta2020', 'totalpobl']], on='uta2020', how='left')
dfs_nac_merged_suphab = dfs_nac_merged_suphab.merge(df_sup_data[['uta2020', 'k2_sup_tot']], on='uta2020', how='left')
# cálculo de la longitud de ruta por superficie y población
dfs_nac_merged_suphab['f_nac_kmsu'] = dfs_nac_merged_suphab['km_nac_tot'] / dfs_nac_merged_suphab['k2_sup_tot']
dfs_nac_merged_suphab['f_nac_kmha'] = dfs_nac_merged_suphab['km_nac_tot'] / dfs_nac_merged_suphab['totalpobl']
dfs_nac_merged_suphab.loc[dfs_nac_merged_suphab.k2_sup_tot == 0, 'f_nac_kmsu'] = 0 
dfs_nac_merged_suphab.loc[dfs_nac_merged_suphab.totalpobl == 0, 'f_nac_kmha'] = 0 
# formateo de los campos numéricos
dfs_nac_merged_suphab.f_nac_kmsu = dfs_nac_merged_suphab.f_nac_kmsu * 100
dfs_nac_merged_suphab.f_nac_kmsu = round(dfs_nac_merged_suphab.f_nac_kmsu, 2)
dfs_nac_merged_suphab.f_nac_kmha = dfs_nac_merged_suphab.f_nac_kmha * 100
dfs_nac_merged_suphab.f_nac_kmha = round(dfs_nac_merged_suphab.f_nac_kmha, 2)
# limpieza de duplicados
dfs_nac_merged_suphab.drop_duplicates(subset=['uta2020'], inplace=True)
dfs_nac_merged_suphab.reset_index(inplace=True, drop=True)

In [26]:
# limpieza del campo UTA2020
dfs_prov_merged.uta2020 = dfs_prov_merged.uta2020.astype(str)
for i in range(len(dfs_prov_merged)):
    if len(dfs_prov_merged.loc[i, 'uta2020']) == 8:
        dfs_prov_merged.loc[i, 'uta2020'] = '0'+dfs_prov_merged.loc[i, 'uta2020']
# unión de la tabla con datos poblacional y la capa de datos de rutas provinciales
dfs_prov_merged_suphab = dfs_prov_merged.merge(df_pob_data[['uta2020', 'totalpobl']], on='uta2020', how='left')
dfs_prov_merged_suphab = dfs_prov_merged_suphab.merge(df_sup_data[['uta2020', 'k2_sup_tot']], on='uta2020', how='left')
# cálculo de la longitud de ruta por superficie y población
dfs_prov_merged_suphab['f_pro_kmsu'] = dfs_prov_merged_suphab['km_pro_tot'] / dfs_prov_merged_suphab['k2_sup_tot']
dfs_prov_merged_suphab['f_pro_kmha'] = dfs_prov_merged_suphab['km_pro_tot'] / dfs_prov_merged_suphab['totalpobl']
dfs_prov_merged_suphab.loc[dfs_prov_merged_suphab.k2_sup_tot == 0, 'f_pro_kmsu'] = 0 
dfs_prov_merged_suphab.loc[dfs_prov_merged_suphab.totalpobl == 0, 'f_pro_kmha'] = 0 
# formateo de los campos numéricos
dfs_prov_merged_suphab.f_pro_kmsu = dfs_prov_merged_suphab.f_pro_kmsu * 100
dfs_prov_merged_suphab.f_pro_kmsu = round(dfs_prov_merged_suphab.f_pro_kmsu, 2)
dfs_prov_merged_suphab.f_pro_kmha = dfs_prov_merged_suphab.f_pro_kmha * 100
dfs_prov_merged_suphab.f_pro_kmha = round(dfs_prov_merged_suphab.f_pro_kmha, 2)
# limpieza de duplicados
dfs_prov_merged_suphab.drop_duplicates(subset=['uta2020'], inplace=True)
dfs_prov_merged_suphab.reset_index(inplace=True, drop=True)

In [27]:
# limpieza del campo UTA2020
dfs_terc_merged.uta2020 = dfs_terc_merged.uta2020.astype(str)
for i in range(len(dfs_terc_merged)):
    if len(dfs_terc_merged.loc[i, 'uta2020']) == 8:
        dfs_terc_merged.loc[i, 'uta2020'] = '0'+dfs_terc_merged.loc[i, 'uta2020']
# unión de la tabla con datos poblacional y la capa de datos de caminos terciarios
dfs_terc_merged_suphab = dfs_terc_merged.merge(df_pob_data[['uta2020', 'totalpobl']], on='uta2020', how='left')
dfs_terc_merged_suphab = dfs_terc_merged_suphab.merge(df_sup_data[['uta2020', 'k2_sup_tot']], on='uta2020', how='left')
# cálculo de la longitud de camino por superficie y población
dfs_terc_merged_suphab['f_ter_kmsu'] = dfs_terc_merged_suphab['km_ter_tot'] / dfs_terc_merged_suphab['k2_sup_tot']
dfs_terc_merged_suphab['f_ter_kmha'] = dfs_terc_merged_suphab['km_ter_tot'] / dfs_terc_merged_suphab['totalpobl']
dfs_terc_merged_suphab.loc[dfs_terc_merged_suphab.k2_sup_tot == 0, 'f_ter_kmsu'] = 0 
dfs_terc_merged_suphab.loc[dfs_terc_merged_suphab.totalpobl == 0, 'f_ter_kmha'] = 0 
# formateo de los campos numéricos
dfs_terc_merged_suphab.f_ter_kmsu = dfs_terc_merged_suphab.f_ter_kmsu * 100
dfs_terc_merged_suphab.f_ter_kmsu = round(dfs_terc_merged_suphab.f_ter_kmsu, 2)
dfs_terc_merged_suphab.f_ter_kmha = dfs_terc_merged_suphab.f_ter_kmha * 100
dfs_terc_merged_suphab.f_ter_kmha = round(dfs_terc_merged_suphab.f_ter_kmha, 2)
# limpieza de duplicados
dfs_terc_merged_suphab.drop_duplicates(subset=['uta2020'], inplace=True)
dfs_terc_merged_suphab.reset_index(inplace=True, drop=True)

In [28]:
# limpieza del campo UTA2020
dfs_urb_merged.uta2020 = dfs_urb_merged.uta2020.astype(str)
for i in range(len(dfs_urb_merged)):
    if len(dfs_urb_merged.loc[i, 'uta2020']) == 8:
        dfs_urb_merged.loc[i, 'uta2020'] = '0'+dfs_urb_merged.loc[i, 'uta2020']
# unión de la tabla con datos poblacional y la capa de datos de caminos urbanos
dfs_urb_merged_suphab = dfs_urb_merged.merge(df_pob_data[['uta2020', 'totalpobl']], on='uta2020', how='left')
dfs_urb_merged_suphab = dfs_urb_merged_suphab.merge(df_sup_data[['uta2020', 'k2_sup_tot']], on='uta2020', how='left')
# cálculo de la longitud de camino por superficie y población
dfs_urb_merged_suphab['f_urb_kmsu'] = dfs_urb_merged_suphab['km_urb_tot'] / dfs_urb_merged_suphab['k2_sup_tot']
dfs_urb_merged_suphab['f_urb_kmha'] = dfs_urb_merged_suphab['km_urb_tot'] / dfs_urb_merged_suphab['totalpobl']
dfs_urb_merged_suphab.loc[dfs_urb_merged_suphab.k2_sup_tot == 0, 'f_urb_kmsu'] = 0 
dfs_urb_merged_suphab.loc[dfs_urb_merged_suphab.totalpobl == 0, 'f_urb_kmha'] = 0 
# formateo de los campos numéricos
dfs_urb_merged_suphab.f_urb_kmsu = dfs_urb_merged_suphab.f_urb_kmsu * 100
dfs_urb_merged_suphab.f_urb_kmsu = round(dfs_urb_merged_suphab.f_urb_kmsu, 2)
dfs_urb_merged_suphab.f_urb_kmha = dfs_urb_merged_suphab.f_urb_kmha * 100
dfs_urb_merged_suphab.f_urb_kmha = round(dfs_urb_merged_suphab.f_urb_kmha, 2)
# limpieza de duplicados
dfs_urb_merged_suphab.drop_duplicates(subset=['uta2020'], inplace=True)
dfs_urb_merged_suphab.reset_index(inplace=True, drop=True)

In [29]:
# eñliminación de las columnas no requeridas
dfs_nac_merged_suphab.drop(columns=['k2_sup_tot', 'totalpobl'], inplace=True)
dfs_prov_merged_suphab.drop(columns=['k2_sup_tot', 'totalpobl'], inplace=True)
dfs_terc_merged_suphab.drop(columns=['k2_sup_tot', 'totalpobl'], inplace=True)
dfs_urb_merged_suphab.drop(columns=['k2_sup_tot', 'totalpobl'], inplace=True)

In [30]:
# exportación de los datos a tablas
dfs_nac_merged_suphab.to_csv("../BD_Procesada/RedVialNacional_GobLocal.csv")
dfs_prov_merged_suphab.to_csv("../BD_Procesada/RedVialProvincial_GobLocal.csv")
dfs_terc_merged_suphab.to_csv("../BD_Procesada/RedVialTerciarias_GobLocal.csv")
dfs_urb_merged_suphab.to_csv("../BD_Procesada/RedVialUrabna_GobLocal.csv")

## Unión con Gob. Locales

In [31]:
# listado de los nombres de las columnas de cada dataframe de rutas
df_columns_rednac = list(dfs_nac_merged_suphab.columns)
df_columns_redprovc = list(dfs_prov_merged_suphab.columns)
df_columns_redterc = list(dfs_terc_merged_suphab.columns)
df_columns_redurb = list(dfs_urb_merged_suphab.columns)
# listado completo de columnas
df_columns_red = df_columns_rednac + df_columns_redprovc[1:] + df_columns_redterc[1:] + df_columns_redurb[1:]

In [32]:
# unión de todas los dataframes de rutas con la capa de gob locales
df_goblocales_rutas = df_goblocales.merge(dfs_nac_merged_suphab, how='left', on="uta2020")
df_goblocales_rutas = df_goblocales_rutas.merge(dfs_prov_merged_suphab, how='left', on="uta2020")
df_goblocales_rutas = df_goblocales_rutas.merge(dfs_terc_merged_suphab, how='left', on="uta2020")
df_goblocales_rutas = df_goblocales_rutas.merge(dfs_urb_merged_suphab, how='left', on="uta2020")
# limpieza de valores nulos
df_goblocales_rutas[['u_nac_tot', 'km_nac_tot', 'km_mna_con', 'km_mna_pav', 'km_mna_tie', 'km_tna_aut', 'km_tna_auv', 'km_tna_rut',
       'f_nac_kmsu', 'f_nac_kmha', 'u_pro_tot', 'km_pro_tot', 'km_mpr_sin', 'km_mpr_pav', 'km_mpr_con', 'km_mpr_tie', 'km_tpr_sin', 'km_tpr_aut',
       'km_tpr_auv', 'km_tpr_cuc', 'km_tpr_cdc', 'f_pro_kmsu', 'f_pro_kmha', 'km_ter_tot', 'km_mte_sin', 'km_mte_pav', 'km_mte_con',
       'km_mte_tie', 'km_tte_sin', 'km_tte_aut', 'km_tte_auv', 'km_tte_cuc', 'km_tte_cdc', 'f_ter_kmsu', 'f_ter_kmha', 'km_urb_tot', 'km_mur_0', 'km_mur_pav', 'km_mur_con',
       'km_mur_tie', 'f_urb_kmsu', 'f_urb_kmha']] = df_goblocales_rutas[['u_nac_tot', 'km_nac_tot', 'km_mna_con', 'km_mna_pav', 'km_mna_tie', 'km_tna_aut', 'km_tna_auv', 'km_tna_rut',
       'f_nac_kmsu', 'f_nac_kmha', 'u_pro_tot', 'km_pro_tot', 'km_mpr_sin', 'km_mpr_pav', 'km_mpr_con', 'km_mpr_tie', 'km_tpr_sin', 'km_tpr_aut',
       'km_tpr_auv', 'km_tpr_cuc', 'km_tpr_cdc', 'f_pro_kmsu', 'f_pro_kmha', 'km_ter_tot', 'km_mte_sin', 'km_mte_pav', 'km_mte_con',
       'km_mte_tie', 'km_tte_sin', 'km_tte_aut', 'km_tte_auv', 'km_tte_cuc', 'km_tte_cdc', 'f_ter_kmsu', 'f_ter_kmha', 'km_urb_tot', 'km_mur_0', 'km_mur_pav', 'km_mur_con',
       'km_mur_tie', 'f_urb_kmsu', 'f_urb_kmha']].fillna(0)

In [33]:
# eliminación de columnas y seteo de los datos de los gob locales sin polígono a "null"
df_goblocales_rutas = df_goblocales_rutas.drop(columns=['area_total_k2'])
df_goblocales_rutas.loc[df_goblocales_rutas['buffer'] == 1, ['st_nac_nam', 'st_pro_nam']] = None
df_goblocales_rutas.loc[df_goblocales_rutas['buffer'] == 1, ['u_nac_tot', 'km_nac_tot', 'km_mna_con', 'km_mna_pav', 'km_mna_tie', 'km_tna_aut', 'km_tna_auv', 'km_tna_rut',
       'f_nac_kmsu', 'f_nac_kmha', 'u_pro_tot', 'km_pro_tot', 'km_mpr_sin', 'km_mpr_pav', 'km_mpr_con', 'km_mpr_tie', 'km_tpr_sin', 'km_tpr_aut',
       'km_tpr_auv', 'km_tpr_cuc', 'km_tpr_cdc', 'f_pro_kmsu', 'f_pro_kmha', 'km_ter_tot', 'km_mte_sin', 'km_mte_pav', 'km_mte_con',
       'km_mte_tie', 'km_tte_sin', 'km_tte_aut', 'km_tte_auv', 'km_tte_cuc', 'km_tte_cdc', 'f_ter_kmsu', 'f_ter_kmha', 'km_urb_tot', 'km_mur_0', 'km_mur_pav', 'km_mur_con',
       'km_mur_tie', 'f_urb_kmsu', 'f_urb_kmha']] = float("NaN")

In [34]:
# exportación de los datos generados a csv y shp
df_goblocales_rutas[df_columns_red].to_csv("../BD_Procesada/MovilidadYTransporteRedesAcceso_GobiernosLocales.csv", encoding='utf-8')
schema_redesviales_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 'tipgobloc':'str', 'uta2020':'str',
                    'u_nac_tot':'int', 'st_nac_nam':'str:2000', 'km_nac_tot':'float:12.3', 'km_mna_con':'float:12.3', 'km_mna_pav':'float:12.3', 'km_mna_tie':'float:12.3',
                    'km_tna_aut':'float:12.3', 'km_tna_auv':'float:12.3', 'km_tna_rut':'float:12.3', 'f_nac_kmsu':'float:10.2', 'f_nac_kmha':'float:10.2',
                    'u_pro_tot':'int', 'st_pro_nam':'str:2000', 'km_pro_tot':'float:12.3', 'km_mpr_sin':'float:12.3', 'km_mpr_pav':'float:12.3', 'km_mpr_con':'float:12.3',
                    'km_mpr_tie':'float:12.3', 'km_tpr_sin':'float:12.3', 'km_tpr_aut':'float:12.3', 'km_tpr_auv':'float:12.3','km_tpr_cuc':'float:12.3',
                    'km_tpr_cdc':'float:12.3', 'f_pro_kmsu':'float:10.2', 'f_pro_kmha':'float:10.2',
                    'km_ter_tot':'float:12.3', 'km_mte_sin':'float:12.3', 'km_mte_pav':'float:12.3', 'km_mte_con':'float:12.3','km_mte_tie':'float:12.3', 'km_tte_sin':'float:12.3',
                    'km_tte_aut':'float:12.3', 'km_tte_auv':'float:12.3', 'km_tte_cuc':'float:12.3', 'km_tte_cdc':'float:12.3','f_ter_kmsu':'float:10.2', 'f_ter_kmha':'float:10.2',
                    'km_urb_tot':'float:12.3', 'km_mur_0':'float:12.3', 'km_mur_pav':'float:12.3', 'km_mur_con':'float:12.3','km_mur_tie':'float:12.3', 
                    'f_urb_kmsu':'float:10.2', 'f_urb_kmha':'float:10.2'}}

df_goblocales_rutas[['provincia', 'departamen', 'municipio', 'tipgobloc']+ df_columns_red+['geometry']].to_file(
    "../CapasGeograficasProcesadas/MovilidadYTransporteRedesAcceso_GobiernosLocales.shp", 
    driver='ESRI Shapefile', 
    schema=schema_redesviales_dic,
    encoding='utf-8')
df_goblocales_rutas.head()

,cpr,provincia,cde,departamen,cgl,municipio,uta2020,indec2019,in1,CodRedatam,...,km_tte_cdc,f_ter_kmsu,f_ter_kmha,km_urb_tot,km_mur_0,km_mur_pav,km_mur_con,km_mur_tie,f_urb_kmsu,f_urb_kmha
0,02,C.A.B.A.,007,Comuna 1,022007,Comuna 1,020072007,nan,020001,20010001,...,5.306,118.45,0.01,296.978,296.978,0.0,0.0,0.0,1660.02,0.14
1,02,C.A.B.A.,014,Comuna 2,022014,Comuna 2,020142014,nan,020002,20020001,...,0.000,113.60,0.00,108.495,108.495,0.0,0.0,0.0,1711.28,0.07
2,02,C.A.B.A.,021,Comuna 3,022021,Comuna 3,020212021,nan,020003,20030001,...,5.082,126.37,0.00,142.542,142.542,0.0,0.0,0.0,2230.70,0.08
3,02,C.A.B.A.,028,Comuna 4,022028,Comuna 4,020282028,nan,020004,20040001,...,0.000,26.72,0.00,442.608,442.608,0.0,0.0,0.0,1944.67,0.20
4,02,C.A.B.A.,035,Comuna 5,022035,Comuna 5,020352035,nan,020005,20050001,...,3.318,132.37,0.00,153.755,153.755,0.0,0.0,0.0,2308.63,0.09


# FFCC

## Estaciones

In [35]:
# filtro de ffcc operativos
df_estFFCC = df_estFFCC.loc[df_estFFCC.fun == 6]
# cruce de las estaciones de FFCC con los gobiernos locales
df_goblocales_estFFCC = gpd.sjoin(df_goblocales, df_estFFCC)
df_goblocales_estFFCC.to_file("../CapasGeograficasCruzadas/estacionesFFCC_goblocales.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """


In [36]:
# Generación de los valores de las distintas variable
# Cantidad de estaciones
df_estFFCC_cant = pd.DataFrame(df_goblocales_estFFCC.value_counts('uta2020')).reset_index()
df_estFFCC_cant.columns=['uta2020', 'est_cantidad']
df_estFFCC_cant.rename(columns={'est_cantidad':'u_tfc_tot'}, inplace=True)
df_estFFCC_cant.head()

,uta2020,u_tfc_tot
0,064270427,21
1,068610861,15
2,064410441,12
3,064900490,10
4,068400840,10


## Redes

In [37]:
# filtro de ffcc operativos
df_redFFCC = df_redFFCC.loc[df_redFFCC.fun == 6]
# cruce de las redes de FFCC con los gobiernos locales
df_inter_gob_FFCC = gpd.overlay(df_redFFCC, df_goblocales, how='union').explode().reset_index(drop=True)
df_inter_gob_FFCC = df_inter_gob_FFCC.loc[df_inter_gob_FFCC.uta2020.notnull()].reset_index()
# cálculo de las líneas de la red
df_inter_gob_FFCC = df_inter_gob_FFCC.to_crs("EPSG:3035")
df_inter_gob_FFCC['long_km'] = df_inter_gob_FFCC.length / 1000
df_inter_gob_FFCC = df_inter_gob_FFCC.to_crs("EPSG:4326")
# exportación de la capa resultante
df_inter_gob_FFCC.to_file("../CapasGeograficasCruzadas/redFFCC_goblocales.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:4: UserWarning: `keep_geom_type=True` in overlay resulted in 2326 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  after removing the cwd from sys.path.
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  after removing the cwd from sys.path.
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  # This is added back by InteractiveShellApp.init_path()


In [38]:
# Generación de los valores de las distintas variable
# Cantidad de redes
df_inter_gob_FFCC_cantidad = df_inter_gob_FFCC.groupby(['uta2020'])['nam'].apply(lambda x: len(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_FFCC_cantidad.rename(columns = {'nam':'FFCC_cant'}, inplace=True)
# Nombres de redes
df_inter_gob_FFCC_name = df_inter_gob_FFCC.groupby(['uta2020'])['nam'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
df_inter_gob_FFCC_name.rename(columns = {'nam':'FFCC_list'}, inplace=True)
df_inter_gob_FFCC_name['FFCC_list'] = df_inter_gob_FFCC_name.apply(lambda row: acortar_lista_nombres(row.FFCC_list), axis=1)
# Longitudes de redes
df_inter_gob_FFCC_long = df_inter_gob_FFCC.groupby(['uta2020'])['long_km'].sum().reset_index()
df_inter_gob_FFCC_long.rename(columns = {'long_km':'FFCC_long_km'}, inplace=True)
# Longitudes de redes por tipo de trocha
df_inter_gob_FFCC_long_rgc = df_inter_gob_FFCC.pivot_table(index='uta2020', columns='rgc', values='long_km', aggfunc=sum).fillna(0).reset_index()
df_inter_gob_FFCC_long_rgc.rename(columns = {1.0:'trocha_ancha', 2.0:'trocha_angosta',
    3.0:'trocha_media', 4.0:'trocha_economica'}, inplace=True)
# merge de todos los dataframes de FFCC
dfs_red_FFCC = [df_inter_gob_FFCC_cantidad, df_inter_gob_FFCC_name, df_inter_gob_FFCC_long, df_inter_gob_FFCC_long_rgc]
dfs_red_FFCC_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_red_FFCC)
dfs_red_FFCC_merged.rename(columns={'FFCC_cant':'u_ffc_tot', 'FFCC_list':'st_ffc_nam', 'FFCC_long_km':'km_ffc_tot', 'trocha_ancha':'km_fct_anc', 'trocha_angosta':'km_fct_ang', 
    'trocha_media':'km_fct_med', 'trocha_economica':'km_fct_eco'}, inplace=True)
# formateo de los campos numéricos
dfs_red_FFCC_merged.km_ffc_tot = round(dfs_red_FFCC_merged.km_ffc_tot, 3)
dfs_red_FFCC_merged.km_fct_anc = round(dfs_red_FFCC_merged.km_fct_anc, 3)
dfs_red_FFCC_merged.km_fct_ang = round(dfs_red_FFCC_merged.km_fct_ang, 3)
dfs_red_FFCC_merged.km_fct_med = round(dfs_red_FFCC_merged.km_fct_med, 3)
dfs_red_FFCC_merged.km_fct_eco = round(dfs_red_FFCC_merged.km_fct_eco, 3)

dfs_red_FFCC_merged.head()

,uta2020,u_ffc_tot,st_ffc_nam,km_ffc_tot,km_fct_anc,km_fct_ang,km_fct_med,km_fct_eco
0,020072007,5,['General Bartolomé Mitre Ramal Suárez/Mitre/T...,15.197,12.237,2.960,0.0,0.0
1,020142014,3,['General Bartolomé Mitre Ramal Suárez/Mitre/T...,6.072,4.031,2.041,0.0,0.0
2,020212021,1,['Domingo Faustino Sarmiento Ramal Lobos/Merce...,0.788,0.788,0.000,0.0,0.0
3,020282028,3,['General Belgrano Sur Ramal González Catán/Ma...,14.544,9.647,4.896,0.0,0.0
4,020352035,1,['Domingo Faustino Sarmiento Ramal Lobos/Merce...,1.573,1.573,0.000,0.0,0.0


## Unión con Gob. Locales

In [39]:
# generación del dataframe de ffcc
dfs_FFCC = [df_estFFCC_cant, dfs_red_FFCC_merged]
dfs_FFCC_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_FFCC)

# listado de columnas
lista_columnas_FFCC = list(dfs_FFCC_merged.columns)

In [40]:
# cruce de los datos generados con los gobienros locales
df_goblocales_FFCC = df_goblocales.merge(dfs_FFCC_merged, on='uta2020', how='left')
df_goblocales_FFCC.u_tfc_tot.fillna(0, inplace=True)
df_goblocales_FFCC.u_ffc_tot.fillna(0, inplace=True)
df_goblocales_FFCC.km_ffc_tot.fillna(0, inplace=True)
df_goblocales_FFCC.km_fct_anc.fillna(0, inplace=True)
df_goblocales_FFCC.km_fct_ang.fillna(0, inplace=True)
df_goblocales_FFCC.km_fct_med.fillna(0, inplace=True)
df_goblocales_FFCC.km_fct_eco.fillna(0, inplace=True)
# eliminación de columnas y seteo de los datos de los gob locales sin polígono a "null"
df_goblocales_FFCC = df_goblocales_FFCC.drop(columns=['area_total_k2'])
df_goblocales_FFCC.loc[df_goblocales_FFCC['buffer'] == 1, 'st_ffc_nam'] = None
df_goblocales_FFCC.loc[df_goblocales_FFCC['buffer'] == 1, ['u_tfc_tot', 'u_ffc_tot', 'km_ffc_tot', 'km_fct_anc', 'km_fct_ang', 
    'km_fct_med', 'km_fct_eco']] = float("NaN")
# exportación de los datos generados a csv y shp
df_goblocales_FFCC[lista_columnas_FFCC].to_csv("../BD_Procesada/MovilidadYTransporteFFCC_GobiernosLocales.csv", encoding='utf-8')
schema_FFCC_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 'tipgobloc':'str', 'uta2020':'str',
                    'u_tfc_tot':'int', 'u_ffc_tot':'int', 'st_ffc_nam':'str:2000', 'km_ffc_tot':'float:12.3',
                    'km_fct_anc':'float:12.3', 'km_fct_ang':'float:12.3', 'km_fct_med':'float:12.3', 'km_fct_eco':'float:12.3'}}
df_goblocales_FFCC[['provincia', 'departamen', 'municipio', 'tipgobloc']+ lista_columnas_FFCC+['geometry']].to_file(
    "../CapasGeograficasProcesadas/MovilidadYTransporteFFCC_GobiernosLocales.shp", 
    driver='ESRI Shapefile', 
    schema=schema_FFCC_dic,
    encoding='utf-8')

df_goblocales_FFCC.head()

,cpr,provincia,cde,departamen,cgl,municipio,uta2020,indec2019,in1,CodRedatam,...,buffer,geometry,u_tfc_tot,u_ffc_tot,st_ffc_nam,km_ffc_tot,km_fct_anc,km_fct_ang,km_fct_med,km_fct_eco
0,02,C.A.B.A.,007,Comuna 1,022007,Comuna 1,020072007,nan,020001,20010001,...,0,"POLYGON ((-58.38609 -34.57823, -58.38592 -34.5...",4.0,5.0,['General Bartolomé Mitre Ramal Suárez/Mitre/T...,15.197,12.237,2.960,0.0,0.0
1,02,C.A.B.A.,014,Comuna 2,022014,Comuna 2,020142014,nan,020002,20020001,...,0,"POLYGON ((-58.38359 -34.57339, -58.38180 -34.5...",1.0,3.0,['General Bartolomé Mitre Ramal Suárez/Mitre/T...,6.072,4.031,2.041,0.0,0.0
2,02,C.A.B.A.,021,Comuna 3,022021,Comuna 3,020212021,nan,020003,20030001,...,0,"POLYGON ((-58.41026 -34.59811, -58.40939 -34.5...",1.0,1.0,['Domingo Faustino Sarmiento Ramal Lobos/Merce...,0.788,0.788,0.000,0.0,0.0
3,02,C.A.B.A.,028,Comuna 4,022028,Comuna 4,020282028,nan,020004,20040001,...,0,"POLYGON ((-58.35517 -34.61945, -58.35511 -34.6...",4.0,3.0,['General Belgrano Sur Ramal González Catán/Ma...,14.544,9.647,4.896,0.0,0.0
4,02,C.A.B.A.,035,Comuna 5,022035,Comuna 5,020352035,nan,020005,20050001,...,0,"POLYGON ((-58.42271 -34.59773, -58.42174 -34.5...",0.0,1.0,['Domingo Faustino Sarmiento Ramal Lobos/Merce...,1.573,1.573,0.000,0.0,0.0


# Terminales 

## Viales

In [41]:
# cruce de las estaciones de omnibus con los gobiernos locales
df_goblocales_estaciones = gpd.sjoin(df_goblocales, df_estaciones)
df_goblocales_estaciones.to_file("../CapasGeograficasCruzadas/estacionesOmnibus_goblocales.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
# Generación de los valores de las distintas variable
# Cantidad de estaciones
df_estaciones_cant = pd.DataFrame(df_goblocales_estaciones.value_counts('uta2020')).reset_index()
df_estaciones_cant.columns=['uta2020', 'est_cantidad']
df_estaciones_cant.rename(columns={'est_cantidad':'u_tom_tot'}, inplace=True)
""" # extracción de los nombres de los campos
df_columns_estaciones = list(df_estaciones_cant.columns)
df_columns_estaciones """
df_estaciones_cant.head()

,uta2020,u_tom_tot
0,064200420,7
1,460420042,2
2,061820182,2
3,501050105,2
4,063990399,2


In [43]:
# generación del dataframe de rutas nacionales
dfs_terminales_merged = df_estaciones_cant

# listado de columnas
lista_columns_terminales = list(dfs_terminales_merged.columns)

# cruce de los datos generados con los gobienros locales
df_movilidad_terminales = df_goblocales.merge(dfs_terminales_merged, on='uta2020', how='left')
df_movilidad_terminales.u_tom_tot.fillna(0, inplace=True)
# eliminación de columnas y seteo de los datos de los gob locales sin polígono a "null"
df_movilidad_terminales = df_movilidad_terminales.drop(columns=['area_total_k2'])
df_movilidad_terminales.loc[df_movilidad_terminales['buffer'] == 1, ['u_tom_tot']] = float("NaN")
# exportación de los datos generados a csv y shp
df_movilidad_terminales[lista_columns_terminales].to_csv("../BD_Procesada/MovilidadYTransporteTerminalesViales_GobiernosLocales.csv", encoding='utf-8')
schema_terminales_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 'tipgobloc':'str', 'uta2020':'str',
                    'u_tom_tot':'int'}}
df_movilidad_terminales[['provincia', 'departamen', 'municipio', 'tipgobloc']+ lista_columns_terminales +['geometry']].to_file(
    "../CapasGeograficasProcesadas/MovilidadYTransporteTerminalesViales_GobiernosLocales.shp", 
    driver='ESRI Shapefile', 
    schema=schema_terminales_dic,
    encoding='utf-8')

## Aéreas

### Aeropuertos

In [44]:
 # limpieza de datos de las capas
df_aerodromo.loc[df_aerodromo.gna == '-1', 'gna'] = 'Aeródromo'
df_aerodromo.loc[df_aerodromo.gna == 'Club de Planeadores', 'gna'] = 'Aeródromo'
df_aerodromo.loc[df_aerodromo.gna == 'Gendarmería Nacional', 'gna'] = 'Aeródromo'
df_aerodromo.loc[df_aerodromo.gna == 'Aerodrómo', 'gna'] = 'Aeródromo'

df_aeropuerto.loc[df_aeropuerto.gna == 'Aeropuerto Regional', 'gna'] = 'Aeropuerto'
df_aeropuerto.loc[df_aeropuerto.gna == 'Aeropuerto Metropolitano', 'gna'] = 'Aeropuerto'
df_aeropuerto.loc[df_aeropuerto.gna == 'Aeroparque', 'gna'] = 'Aeropuerto'
df_aeropuerto['atv'] = 9

df_aero = pd.concat([df_aerodromo, df_aeropuerto]).reset_index(drop=True)


In [45]:
# cruce de los aerpuertos con la capa de gob locales
df_goblocales_aero = gpd.sjoin(df_goblocales, df_aero)
df_goblocales_aero.loc[df_goblocales_aero.gna.isnull(), 'gna'] = df_goblocales_aero.loc[df_goblocales_aero.gna.isnull(), 'objeto']
df_goblocales_aero.to_file("../CapasGeograficasCruzadas/aeropuertos_goblocales.shp")

# Generación de los valores de las distintas variable
# Cantidad de aeropuertos
df_goblocales_aero_canttotal = pd.DataFrame(df_goblocales_aero.value_counts('uta2020')).reset_index()
df_goblocales_aero_canttotal.columns = ['uta2020', 'aero_cantidad']
# Nombre de aeropuertos
df_goblocales_aero_name = df_goblocales_aero.loc[df_goblocales_aero.nam.notnull()].groupby(['uta2020'])['fna'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
df_goblocales_aero_name.rename(columns = {'fna':'aero_list'}, inplace=True)
df_goblocales_aero_name['aero_list'] = df_goblocales_aero_name.apply(lambda row: acortar_lista_nombres(row.aero_list), axis=1)
# Cantidad de aeropuertos por tipo
df_goblocales_aero_cant_tipo = df_goblocales_aero.pivot_table(index='uta2020', columns='gna', aggfunc='size', fill_value=0).reset_index()

# merge de todos los dataframes de aeropuertos
dfs_aero = [df_goblocales_aero_canttotal, df_goblocales_aero_name, df_goblocales_aero_cant_tipo]
dfs_aero_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_aero)
dfs_aero_merged.rename(columns={'aero_cantidad':'u_aer_tot', 'aero_list':'st_aer_nam', 'Aeroclub':'u_clu_tot', 'Aeropuerto':'u_pue_tot', 'Aeropuerto Internacional':'u_pui_tot', 
    'Aeródromo':'u_dro_tot'}, inplace=True)
dfs_aero_merged.head()

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  after removing the cwd from sys.path.


,uta2020,u_aer_tot,st_aer_nam,u_clu_tot,u_pue_tot,u_pui_tot,u_dro_tot
0,064690469,13,"['Aeródromo Lincoln', 'Aeródromo Fumigaciones ...",0,0,0,13
1,181050245,8,"['Aeródromo Estancia Rincón de San José', 'Aer...",0,0,0,8
2,660070028,8,"['Aeródromo SAS Aeroaplicaciones', '...', 'Aer...",0,0,0,8
3,068540854,7,"['Aeródromo Estancia El Pajonal', 'Aeródromo 2...",0,0,0,7
4,063920392,7,"['Aeródromo San Eduardo', 'Aeródromo Estancia ...",0,0,0,7


### Helipuertos

In [46]:
# cruce de los helipuertos con la capa de gob locales
df_goblocales_heli = gpd.sjoin(df_goblocales, df_helipuerto)
df_goblocales_heli.to_file("../CapasGeograficasCruzadas/helipuertos_goblocales.shp")

# Generación de los valores de las distintas variable
# Cantidad de helipuertos
df_goblocales_heli_canttotal = pd.DataFrame(df_goblocales_heli.value_counts('uta2020')).reset_index()
df_goblocales_heli_canttotal.columns = ['uta2020', 'heli_cantidad']
# Nombre de helipuertos
df_goblocales_heli_name = df_goblocales_heli.loc[df_goblocales_heli.nam.notnull()].groupby(['uta2020'])['fna'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
df_goblocales_heli_name.rename(columns = {'fna':'heli_list'}, inplace=True)
df_goblocales_heli_name['heli_list'] = df_goblocales_heli_name.apply(lambda row: acortar_lista_nombres(row.heli_list), axis=1)
# Cantidad de helipuertos por tipo
df_goblocales_heli_cant_tipo = df_goblocales_heli.pivot_table(index='uta2020', columns='gna', aggfunc='size', fill_value=0).reset_index()
# merge de todos los dataframes de helipuertos
dfs_heli = [df_goblocales_heli_canttotal, df_goblocales_heli_name, df_goblocales_heli_cant_tipo]
dfs_heli_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_heli)
dfs_heli_merged.rename(columns={'heli_cantidad':'u_hel_tot', 'heli_list':'st_hel_nam'}, inplace=True)
dfs_heli_merged.drop(columns='Helipuerto', inplace=True)
dfs_heli_merged.head()

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


,uta2020,u_hel_tot,st_hel_nam
0,068050805,7,"['Helipuerto Tecnocopter S.A.', 'Helipuerto Vi..."
1,020282028,4,"['Helipuerto Dársena Sur', 'Helipuerto Complej..."
2,064270427,4,"['Helipuerto Campanópolis', 'Helipuerto Oficia..."
3,140140077,3,"['Helipuerto Policía Provincial de Córdoba', '..."
4,620210077,3,['Helipuerto San Carlos de Bariloche / Complej...


### Unión con Gob. Locales

In [47]:
# generación del dataframe de rutas nacionales
dfs_terminales = [dfs_aero_merged, dfs_heli_merged]
dfs_terminales_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_terminales)

# listado de columnas
lista_columns_terminales = list(dfs_terminales_merged.columns)

# cruce de los datos generados con los gobienros locales
df_movilidad_terminales = df_goblocales.merge(dfs_terminales_merged, on='uta2020', how='left')
df_movilidad_terminales.u_aer_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_clu_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_pue_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_pui_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_dro_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_hel_tot.fillna(0, inplace=True)
# eliminación de columnas y seteo de los datos de los gob locales sin polígono a "null"
df_movilidad_terminales = df_movilidad_terminales.drop(columns=['area_total_k2'])
df_movilidad_terminales.loc[df_movilidad_terminales['buffer'] == 1, ['st_aer_nam', 'st_hel_nam']] = None
df_movilidad_terminales.loc[df_movilidad_terminales['buffer'] == 1, ['u_aer_tot', 'u_clu_tot', 'u_pue_tot', 'u_pui_tot', 'u_dro_tot', 
    'u_hel_tot', 'u_pna_tot', 'u_tbn_tot', 'u_tpn_tot']] = float("NaN")
# exportación de los datos generados a csv y shp
df_movilidad_terminales[lista_columns_terminales].to_csv("../BD_Procesada/MovilidadYTransporteTerminalesAereas_GobiernosLocales.csv", encoding='utf-8')
schema_terminales_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 'tipgobloc':'str', 'uta2020':'str',
                    'u_aer_tot':'int', 'st_aer_nam':'str:2000', 'u_clu_tot':'int', 'u_pue_tot':'int', 'u_pui_tot':'int', 'u_dro_tot':'int',
                    'u_hel_tot':'int', 'st_hel_nam':'str:2000'}}
df_movilidad_terminales[['provincia', 'departamen', 'municipio', 'tipgobloc']+ lista_columns_terminales +['geometry']].to_file(
    "../CapasGeograficasProcesadas/MovilidadYTransporteTerminalesAereas_GobiernosLocales.shp", 
    driver='ESRI Shapefile', 
    schema=schema_terminales_dic,
    encoding='utf-8')

## Fluvial

In [48]:
df_puertos.loc[df_puertos.gna == 'Centro de Embarque', 'gna'] = 'Puerto'
df_puertos.loc[df_puertos.gna == 'Club Náutico', 'gna'] = 'Puerto'
df_puertos.loc[df_puertos.gna == 'Terminal', 'gna'] = 'Puerto'

In [49]:
# cruce de los puertos con la capa de gob locales
df_goblocales_puerto = gpd.sjoin(df_goblocales, df_puertos)
df_goblocales_puerto.to_file("../CapasGeograficasCruzadas/puertos_goblocales.shp")

# Generación de los valores de las distintas variable
# Cantidad de puertos
df_goblocales_puerto_canttotal = pd.DataFrame(df_goblocales_puerto.value_counts('uta2020')).reset_index()
df_goblocales_puerto_canttotal.columns = ['uta2020', 'puerto_cantidad']
# Nombre de puertos
df_goblocales_puerto_name = df_goblocales_puerto.loc[df_goblocales_puerto.nam.notnull()].groupby(['uta2020'])['fna'].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
df_goblocales_puerto_name.rename(columns = {'fna':'puerto_list'}, inplace=True)
df_goblocales_puerto_name['puerto_list'] = df_goblocales_puerto_name.apply(lambda row: acortar_lista_nombres(row.puerto_list), axis=1)
# Cantidad de puertos por tipo
df_goblocales_puerto_cant_tipo = df_goblocales_puerto.pivot_table(index='uta2020', columns='gna', aggfunc='size', fill_value=0).reset_index()
# merge de todos los dataframes de puertos
dfs_puerto = [df_goblocales_puerto_canttotal, df_goblocales_puerto_name, df_goblocales_puerto_cant_tipo]
dfs_puerto_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_puerto)
dfs_puerto_merged.rename(columns={'puerto_cantidad':'u_pna_tot', 'puerto_list':'st_pna_nam', 'Base Naval':'u_tbn_tot', 'Puerto':'u_tpn_tot'}, inplace=True)
dfs_puerto_merged.head()

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


,uta2020,u_pna_tot,st_pna_nam,u_tbn_tot,u_tpn_tot
0,821190287,10,['Puerto ADM Agro S.R.L. (Terminal El Tránsito...,0,10
1,061260126,9,"['Puerto Petromining. S.A.', '...', 'Puerto Eu...",0,9
2,068820882,8,"['Puerto Molca S.A.', 'Terminal Zárate S.A.', ...",0,8
3,821190301,7,['Puerto Asociación de Cooperativas Argentinas...,0,7
4,620210077,6,"['Puerto Alegre', 'Puerto Blest', 'Club Náutic...",0,6


In [50]:
# generación del dataframe de rutas nacionales
dfs_terminales_merged = dfs_puerto_merged

# listado de columnas
lista_columns_terminales = list(dfs_terminales_merged.columns)

# cruce de los datos generados con los gobienros locales
df_movilidad_terminales = df_goblocales.merge(dfs_terminales_merged, on='uta2020', how='left')
df_movilidad_terminales.u_pna_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_tbn_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_tpn_tot.fillna(0, inplace=True)
# eliminación de columnas y seteo de los datos de los gob locales sin polígono a "null"
df_movilidad_terminales = df_movilidad_terminales.drop(columns=['area_total_k2'])
df_movilidad_terminales.loc[df_movilidad_terminales['buffer'] == 1, ['st_pna_nam']] = None
df_movilidad_terminales.loc[df_movilidad_terminales['buffer'] == 1, ['u_pna_tot', 'u_tbn_tot', 'u_tpn_tot']] = float("NaN")
# exportación de los datos generados a csv y shp
df_movilidad_terminales[lista_columns_terminales].to_csv("../BD_Procesada/MovilidadYTransporteTerminalesFluviales_GobiernosLocales.csv", encoding='utf-8')
schema_terminales_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 'tipgobloc':'str', 'uta2020':'str',
                    'u_pna_tot':'int', 'st_pna_nam':'str:2000', 'u_tbn_tot':'int', 'u_tpn_tot':'int'}}
df_movilidad_terminales[['provincia', 'departamen', 'municipio', 'tipgobloc']+ lista_columns_terminales +['geometry']].to_file(
    "../CapasGeograficasProcesadas/MovilidadYTransporteTerminalesFluviales_GobiernosLocales.shp", 
    driver='ESRI Shapefile', 
    schema=schema_terminales_dic,
    encoding='utf-8')

## Unión con Gob. Locales

In [51]:
""" # generación del dataframe de rutas nacionales
dfs_terminales = [df_estaciones_cant, dfs_aero_merged, dfs_heli_merged, dfs_puerto_merged]
dfs_terminales_merged = reduce(lambda  left,right: pd.merge(left,right,on=['uta2020'], how='outer'), dfs_terminales)

# listado de columnas
lista_columns_terminales = list(dfs_terminales_merged.columns)

# cruce de los datos generados con los gobienros locales
df_movilidad_terminales = df_goblocales.merge(dfs_terminales_merged, on='uta2020', how='left')
df_movilidad_terminales.u_tom_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_aer_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_clu_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_pue_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_pui_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_dro_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_hel_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_pna_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_tbn_tot.fillna(0, inplace=True)
df_movilidad_terminales.u_tpn_tot.fillna(0, inplace=True)
# eliminación de columnas y seteo de los datos de los gob locales sin polígono a "null"
df_movilidad_terminales = df_movilidad_terminales.drop(columns=['area_total_k2'])
df_movilidad_terminales.loc[df_movilidad_terminales['buffer'] == 1, ['st_aer_nam', 'st_hel_nam', 'st_pna_nam']] = None
df_movilidad_terminales.loc[df_movilidad_terminales['buffer'] == 1, ['u_tom_tot', 'u_aer_tot', 'u_clu_tot', 'u_pue_tot', 'u_pui_tot', 'u_dro_tot', 
    'u_hel_tot', 'u_pna_tot', 'u_tbn_tot', 'u_tpn_tot']] = float("NaN")
# exportación de los datos generados a csv y shp
df_movilidad_terminales[lista_columns_terminales].to_csv("../BD_Procesada/MovilidadYTransporteTerminales_GobiernosLocales.csv", encoding='utf-8')
schema_terminales_dic={'geometry': 'Polygon', 'properties': {'provincia':'str', 'departamen':'str', 'municipio':'str', 'tipgobloc':'str', 'uta2020':'str',
                    'u_tom_tot':'int', 'u_aer_tot':'int', 'st_aer_nam':'str:2000', 'u_clu_tot':'int', 'u_pue_tot':'int', 'u_pui_tot':'int', 'u_dro_tot':'int',
                    'u_hel_tot':'int', 'st_hel_nam':'str:2000', 'u_pna_tot':'int', 'st_pna_nam':'str:2000', 
                    'u_tbn_tot':'int', 'u_tpn_tot':'int'}}
df_movilidad_terminales[['provincia', 'departamen', 'municipio', 'tipgobloc']+ lista_columns_terminales +['geometry']].to_file(
    "../CapasGeograficasProcesadas/MovilidadYTransporteTerminales_GobiernosLocales.shp", 
    driver='ESRI Shapefile', 
    schema=schema_terminales_dic,
    encoding='utf-8') """

' # generación del dataframe de rutas nacionales\ndfs_terminales = [df_estaciones_cant, dfs_aero_merged, dfs_heli_merged, dfs_puerto_merged]\ndfs_terminales_merged = reduce(lambda  left,right: pd.merge(left,right,on=[\'uta2020\'], how=\'outer\'), dfs_terminales)\n\n# listado de columnas\nlista_columns_terminales = list(dfs_terminales_merged.columns)\n\n# cruce de los datos generados con los gobienros locales\ndf_movilidad_terminales = df_goblocales.merge(dfs_terminales_merged, on=\'uta2020\', how=\'left\')\ndf_movilidad_terminales.u_tom_tot.fillna(0, inplace=True)\ndf_movilidad_terminales.u_aer_tot.fillna(0, inplace=True)\ndf_movilidad_terminales.u_clu_tot.fillna(0, inplace=True)\ndf_movilidad_terminales.u_pue_tot.fillna(0, inplace=True)\ndf_movilidad_terminales.u_pui_tot.fillna(0, inplace=True)\ndf_movilidad_terminales.u_dro_tot.fillna(0, inplace=True)\ndf_movilidad_terminales.u_hel_tot.fillna(0, inplace=True)\ndf_movilidad_terminales.u_pna_tot.fillna(0, inplace=True)\ndf_movilidad_te